# Multi-tenant Chat with Papers - Setup
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
openai_url = os.getenv("OPENAI_URL")

print(weaviate_url, openai_url)

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    host=weaviate_url
)

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Papers")):
    client.collections.delete("Papers")

client.collections.create(
    "Papers",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            model="text-embedding-3-small",
            base_url=openai_url,

            # we don't need source_properties in this example - unless we expect to add data without providing vectors
            # source_properties=['title', 'text']
        )
    ],

    generative_config=Configure.Generative.openai(
        model="gpt-4",
        base_url=openai_url
    ),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## List Tenants

In [ ]:
papers = client.collections.get("Papers")
papers.tenants.get()

## Close the client

In [ ]:
client.close()